This notebook focuses on understanding the Spacex DataSet by loading the dataset  into the corresponding table in a Db2 database and executing SQL queries.


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

(.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [1]:
!pip install sqlalchemy==1.3.9


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 72.8 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.9-cp37-cp37m-linux_x86_64.whl size=1159121 sha256=9c5058ea2da3b377a467eb196caed7c2cbbb4356f7685eb8fae6f871b4e8dc12
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/03/71/13/010faf12246f72dc76b4150e6e599d13a85b4435e06fb9e51f
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.24
    Uninstalling SQLAlchemy-1.3.24:
      Successfully uninstalled SQLAlchemy-1.3.24


Let us first load the SQL extension and establish a connection with the database


In [2]:
%load_ext sql

In [3]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
!pip install -q pandas==1.1.5

In [5]:
%sql sqlite:///my_data1.db

'Connected: @my_data1.db'

In [6]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  both result in 0.1234 being formatted as 0.12.


##### Displaying the names of the unique launch sites  in the space mission


In [7]:
%%sql
SELECT DISTINCT "Launch_Site" FROM SPACEXTBL

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40
None


#####  Displaying 5 records where launch sites begin with the string 'CCA' 


In [8]:
%%sql
SELECT DISTINCT "Launch_Site" FROM SPACEXTBL
WHERE "Launch_Site" LIKE 'CCA%' LIMIT 5

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
CCAFS SLC-40


##### Displaying the total payload mass carried by boosters launched by NASA (CRS)


In [9]:
%%sql
SELECT SUM(PAYLOAD_MASS__KG_) FROM SPACEXTBL
WHERE "Customer"="NASA (CRS)"

 * sqlite:///my_data1.db
Done.


SUM(PAYLOAD_MASS__KG_)
45596.0


##### Displaying average payload mass carried by booster version F9 v1.1


In [10]:
%%sql

SELECT AVG(PAYLOAD_MASS__KG_) FROM SPACEXTBL
WHERE "Booster_Version" LIKE "F9 v1.1%"

 * sqlite:///my_data1.db
Done.


AVG(PAYLOAD_MASS__KG_)
2534.6666666666665


##### When was the first succesful landing outcome in ground pad ?


In [11]:
%%sql

SELECT MIN(Date) FROM SPACEXTBL
WHERE "Mission_Outcome"="Success" 

 * sqlite:///my_data1.db
Done.


MIN(Date)
01/06/2014


##### Listing the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [17]:
%%sql

SELECT DISTINCT "Booster_Version" FROM SPACEXTBL
WHERE "Landing_Outcome" LIKE "Success%"
AND "Landing_Outcome" LIKE "%drone ship%"
AND PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


##### Listing the total number of successful and failure mission outcomes


In [18]:
%%sql
SELECT COUNT(*),"Mission_Outcome" 
FROM SPACEXTBL GROUP BY "Mission_Outcome"

 * sqlite:///my_data1.db
Done.


COUNT(*),Mission_Outcome
898,None
1,Failure (in flight)
98,Success
1,Success
1,Success (payload status unclear)


##### Listing the names of the booster_versions which have carried the maximum payload mass.


In [19]:
%%sql
SELECT DISTINCT "Booster_Version" FROM SPACEXTBL
WHERE PAYLOAD_MASS__KG_ = (
SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTBL)

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


##### Listing the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

Note: SQLLite does not support monthnames. So you need to use  substr(Date, 4, 2) as month to get the months and substr(Date,7,4)='2015' for year.


In [21]:
%%sql
SELECT substr(Date, 4, 2) AS MONTH,"Landing _Outcome","Booster_Version","Launch_Site"
FROM SPACEXTBL WHERE SUBSTR(Date,7,4)='2015' AND 
"Landing_Outcome" LIKE "Failure%" AND
"Landing_Outcome" LIKE "%drone ship%" 

 * sqlite:///my_data1.db
Done.


MONTH,"""Landing _Outcome""",Booster_Version,Launch_Site
10,Landing _Outcome,F9 v1.1 B1012,CCAFS LC-40
04,Landing _Outcome,F9 v1.1 B1015,CCAFS LC-40


##### Rank the  count of  successful landing_outcomes between the date 04-06-2010 and 20-03-2017 in descending order.


In [23]:
%%sql
SELECT "Landing_Outcome", COUNT(*) AS Count FROM SPACEXTBL 
WHERE Date BETWEEN '04-06-2010' and '20-03-2017'
GROUP BY "Landing_Outcome"
HAVING "Landing_Outcome" LIKE "Success%"
ORDER BY Count DESC

 * sqlite:///my_data1.db
Done.


Landing_Outcome,Count
Success,20
Success (drone ship),8
Success (ground pad),7


### Reference Links

* <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>  

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*  <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb">Hands-on Lab: Analyzing a real World Data Set</a>




## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
